### FE HW3

In [ ]:
import numpy as np
import pandas as pd

### One step running

In [ ]:
def cal_payoff(z):
    r = z.dot(A)
    columns = []
    for i in range(n):
        columns.append('r'+str(i+1))
    df = pd.DataFrame(r, columns = columns)

    mu = np.zeros(n)
    col_name = []
    for i in range(n):
        idx = str(i+1)
        col_name.append('ST_'+idx)
        mu[i] = np.log(d['S'+idx+'0']) + (d['r'] - d['q'+idx] - (d['sigma'+idx] ** 2)/2) * d['T'] # mu = ln(S_i0) + (r_i - q_i -sigma_i^2/2)*T
        df['ln(ST_'+idx+')'] = df['r'+idx] + mu[i] # shift mean
        df['ST_' + idx] = np.exp(df['ln(ST_'+idx+')']) # take exp
    df['zero'] = 0
    df['max-K'] = df[col_name].max(axis=1) - d['K']
    df['Payoff'] = df[['max-K', 'zero']].max(axis=1)
    return df['Payoff'].mean() * np.exp(-d['r'] * d['T'])

In [ ]:
def simulation():
    mean= np.zeros(3)
    std = np.zeros(3)
    val_arr = np.zeros(d['rep_num'])
    val_1_arr = np.zeros(d['rep_num'])
    val_2_arr = np.zeros(d['rep_num'])
    for rep in range(d['rep_num']):
        z = np.zeros((d['sim_num'], n))
        z1 = np.zeros((d['sim_num'], n))
        for i in range(n):
            z[:,i] = np.random.normal(0, 1, d['sim_num'])

        z1[:int(d['sim_num']/2)] = z[:int(d['sim_num']/2)]
        z1[int(d['sim_num']/2):] = -z[:int(d['sim_num']/2)]
        for i in range(n):
            z1[:,i] = z1[:,i]/z1[:,i].std()

        C_hat = pd.DataFrame(z1).cov().values
        B = Cholesky_decomposition(C_hat)
        z2 = z1.dot(np.linalg.inv(B))
        C_new = pd.DataFrame(z2).cov().values
        #print(C_new)
        val_arr[rep] = cal_payoff(z)
        val_1_arr[rep] = cal_payoff(z1)
        val_2_arr[rep] = cal_payoff(z2)

    mean[0], std[0] = val_arr.mean(), val_arr.std()
    mean[1], std[1] = val_1_arr.mean(), val_1_arr.std()
    mean[2], std[2] = val_2_arr.mean(), val_2_arr.std()
    return mean, std, C_hat

In [ ]:
def Cholesky_decomposition(mat_C):
    A = np.zeros((n,n))
    # Step 1
    A[0][0] = np.sqrt(mat_C[0][0])
    #print("mat_C[0][0] = ", mat_C[0][0])
    for j in range(n):
        A[0][j] = mat_C[0][j]/A[0][0]

    # Step 2
    for i in range(1,n-1):
        temp = mat_C[i][i]
        for k in range(i):
            temp -= (A[k][i] ** 2)
        A[i][i] = np.sqrt(temp)
        #print("temp in step 2 = ", temp)

        # Step 3
        for j in range(i+1, n):
            temp = mat_C[i][j]
            for k in range(i):
                temp -= A[k][i] * A[k][j]
            A[i][j] = temp/A[i][i]

    # Step 4
    temp = mat_C[n-1][n-1]
    #print("---------------------")
    #print("start_temp in step 4 = ", temp)
    for k in range(n-1):
        temp -= (A[k][n-1] **2)
        #print("K = ", k, ', temp = ', temp)
    A[n-1][n-1] = np.sqrt(round(temp, 12))
    #print("end_temp in step 4 = ", temp)
    #print("---------------------")
    return A

In [ ]:
# Read files
d = {}

with open('HW3_param_3.txt','r') as file:
    line = file.readline().strip('\n')
    while line:
        key, value = line.split('=')[0], float(line.split('=')[1])
        d[key] = value
        line = file.readline().strip('\n')

d['sim_num'] = int(d['sim_num'])
d['rep_num'] = int(d['rep_num'])
d['n'] = int(d['n'])
# print("Parameters:\n", d)

n = d['n']
rho_matrix = np.zeros((n,n))
sigma_arr = np.zeros(n)
for i in range(n):
    sigma_arr[i] = d['sigma'+str(i+1)]
    for j in range(n):
        if i==j:
            rho_matrix[i][j] = 1
        elif i<j:
            rho_matrix[i][j] = d['rho'+str(i+1)+str(j+1)] # eg: rho12
        else:
            rho_matrix[i][j] = d['rho'+str(j+1)+str(i+1)]

# Build C matrix
C = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        C[i][j] = rho_matrix[i][j] * sigma_arr[i] * sigma_arr[j] * d['T']
# print("matrix C:\n:", C)
# Build A matrix
A = Cholesky_decomposition(C)
# print("matrix A:\n", A)
# Simulate
mean, std, C_hat = simulation()
# print("std in three cases: ", std[0], std[1], std[2])
# print(std[1]>std[2])
for i in range(3):
    if i==0 :
        print("Basic requirement: ", end='')
    else:
        print("Bonus " + str(i) + ":           ", end='')
    print("Option value = %6f and 95 percent C.I. = [%6f, %6f]" % (mean[i], mean[i]-2*std[i], mean[i]+2*std[i]))

Basic requirement: Option value = 30.257542 and 95 percent C.I. = [29.552744, 30.962340]
Bonus 1:           Option value = 30.409941 and 95 percent C.I. = [30.220627, 30.599256]
Bonus 2:           Option value = 30.399410 and 95 percent C.I. = [30.242108, 30.556712]
